# Create DeepAR+ predcitors with HPO

* **About 90 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# 다른 노트북에서 저장된 변수들을 불러옴
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Hyperparameters

In [4]:
forecastHorizon = 31 # the number of forecasts
NumberOfBacktestWindows = 2 # the number of backtests
BackTestWindowOffset = 31 
ForecastFrequency = "D"

The detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html) 를 보세요.

#### Predictor algorithms

In [5]:
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### DeepAR+

In [6]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_HPO' + suffix

In [7]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=True,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )





## Check if the predictors are active


In [8]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [9]:
target_deepar_hpo_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [10]:
%%time
# Check the Prophet status

while True:
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_hpo_predictorArn)['Status']    
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

DeepARP:  CREATE_PENDING
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
DeepARP:  CREATE_I

In [13]:
# 90 mins elapsed
forecast.describe_predictor(PredictorArn= target_deepar_hpo_predictorArn)   

{'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/StoreItemDemand_deeparp_algo_HPO47645',
 'PredictorName': 'StoreItemDemand_deeparp_algo_HPO47645',
 'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
 'ForecastHorizon': 31,
 'PerformAutoML': False,
 'PerformHPO': True,
 'TrainingParameters': {'context_length': '196',
  'epochs': '500',
  'learning_rate': '0.003123529019933621',
  'learning_rate_decay': '0.5',
  'likelihood': 'student-t',
  'max_learning_rate_decays': '0',
  'num_averaged_models': '1',
  'num_cells': '40',
  'num_layers': '2',
  'prediction_length': '31'},
 'EvaluationParameters': {'NumberOfBacktestWindows': 2,
  'BackTestWindowOffset': 31},
 'InputDataConfig': {'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/StoreItemDemandDSG_target47645',
  'SupplementaryFeatures': [{'Name': 'holiday', 'Value': 'US'}]},
 'FeaturizationConfig': {'ForecastFrequency': 'D',
  'ForecastDimensions': ['store'],
  'Featurizations': [{'At

## Predictor evaluation

#### DeepAR+

Same as Prophet and ARIMA, now you should look at the metrics from it.

In [11]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_hpo_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 9.473244457476685,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05528602179939943,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.12314987504889768,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.053503202816914855,
                                                                                          'Quantile': 0.1}]}},
                                      

In [12]:
%store target_deepar_hpo_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
